# Dogs vs Cats

## Загрузка датасета из Kaggle

In [1]:
!make

pip install kaggle
mkdir ~/.kaggle
mkdir: cannot create directory ‘/root/.kaggle’: File exists
make: *** [Makefile:7: dir] Error 1


In [1]:
import kaggle

In [2]:
kaggle.api.authenticate()

In [3]:
kaggle.api.competition_download_files('dogs-vs-cats-redux-kernels-edition', path='datas')

## Распаковка zip-архивов

In [ ]:
import zipfile
import os

os.makedirs('datas/extracted', exist_ok=True)

with zipfile.ZipFile('datas/dogs-vs-cats-redux-kernels-edition.zip', 'r') as zip_ref:
        zip_ref.extractall('datas/extracted')

In [ ]:
with zipfile.ZipFile('datas/extracted/train.zip','r') as zip_ref:
        zip_ref.extractall('datas/extracted')

with zipfile.ZipFile('datas/extracted/test.zip','r') as zip_ref:
        zip_ref.extractall('datas/extracted')

## Импорт необоходимых библиотек

In [14]:
import os
from random import shuffle
from glob import glob

import cv2
import numpy as np
import tensorflow as tf 
from tensorflow.keras.applications import vgg16

print(tf.__version__)
print(tf.executing_eagerly())

2.18.0
True


## Функции загрузки данных

In [ ]:
IMG_SIZE = (224, 224)  # размер входного изображения сети

train_files = glob('datas/train/*.jpg')
test_files = glob('data/test_sample/*.jpg')

# загружаем входное изображение и предобрабатываем
def load_image(path, target_size=IMG_SIZE):
    img = cv2.imread(path)[...,::-1]
    img = cv2.resize(img, target_size)
    return vgg16.preprocess_input(img)  # предобработка для VGG16

# функция-генератор загрузки обучающих данных с диска
def fit_generator(files, batch_size=32):
    batch_size = min(batch_size, len(files))
    while True:
        shuffle(files)
        for k in range(len(files) // batch_size):
            i = k * batch_size
            j = i + batch_size
            if j > len(files):
                j = - j % len(files)
            x = np.array([load_image(path) for path in files[i:j]])
            y = np.array([1. if os.path.basename(path).startswith('dog') else 0.
                          for path in files[i:j]])
            yield (x, y)

# функция-генератор загрузки тестовых изображений с диска
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])